# Inclusive ML - Understanding Bias

**Learning Objectives**

In this lab, you use a Juypter Notebook to:
- Invoke the What-if Tool against a deployed Model
- Explore attributes of the dataset
- Examine aspects of bias in model results
- Evaluate how the What-if Tool provides suggestions to remediate bias

---
## Introduction 

This notebook shows use of the [What-If Tool](https://pair-code.github.io/what-if-tool) inside of a Jupyter notebook.  The What-If Tool, among many other things, allows us to explore the impacts of Fairness in model design and deployment.

The notebook invokes a deployed XGBoost classifier model on the [UCI census dataset](https://archive.ics.uci.edu/ml/datasets/census+income) which predicts whether a person earns more than $50K based on their census information.

You will then visualize the results of the trained classifier on test data using the What-If Tool.  
___

Let's import the libraries needed for the lab.

In [1]:
pip freeze | grep witwidget || pip install witwidget

witwidget==1.4.2
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import warnings

import numpy as np
import pandas as pd
import witwidget
from witwidget.notebook.visualization import (
    WitWidget, 
    WitConfigBuilder,
)

In [18]:
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

In the next cell, replace `<YOUR PROJEC>` by your GCP project id:

In [19]:
PROJECT = "<YOUR PROJEC>"

In [25]:
BUCKET = "gs://{project}".format(project=PROJECT)
MODEL = 'xgboost_model'
VERSION = 'v1'
MODEL_DIR = os.path.join(BUCKET, MODEL)


os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['MODEL'] = MODEL
os.environ['VERSION'] = VERSION
os.environ['MODEL_DIR'] = MODEL_DIR

### Set up the notebook environment

First you must perform a few environment and project configuration steps.  


These steps may take 8 - 10 minutes, please wait until you see the following response before proceeding:

"__Creating version (this might take a few minutes)......done.__"

In [30]:
%%bash

gcloud config set project $PROJECT

gsutil ls $BUCKET || gsutil mb $BUCKET
gsutil cp gs://cloud-training-demos/mlfairness/model.bst $MODEL_DIR/model.bst

gcloud ai-platform models list | grep $MODEL || gcloud ai-platform models create $MODEL

gcloud ai-platform versions list --model $MODEL | grep $VERSION ||
gcloud ai-platform versions create $VERSION \
  --model=$MODEL \
  --framework='XGBOOST' \
  --runtime-version=1.14 \
  --origin=$MODEL_DIR \
  --python-version=3.5 \
  --project=$PROJECT

gs://dherin-sandbox/2fc645062bfd281c454046b0874f5fd79e8b9f7bc601be3196e270ed3fc2cb7a/
gs://dherin-sandbox/4e5c2dffdbaae7a20248b858a136de165fe4c22fad881716f61e50415282e220/
gs://dherin-sandbox/datasets/
gs://dherin-sandbox/taxifare/
gs://dherin-sandbox/text2hub/
gs://dherin-sandbox/xgboost_model/
xgboost_model  v1
v1    gs://dherin-sandbox/xgboost_model  READY


Updated property [core/project].
Copying gs://cloud-training-demos/mlfairness/model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 65.2 KiB/ 65.2 KiB]                                                
Operation completed over 1 objects/65.2 KiB.                                     


---
Finally download the data and arrays needed to use the What-if Tool.

In [31]:
%%bash

gsutil cp gs://cloud-training-demos/mlfairness/income.pkl .
gsutil cp gs://cloud-training-demos/mlfairness/x_test.npy .
gsutil cp gs://cloud-training-demos/mlfairness/y_test.npy .  

Copying gs://cloud-training-demos/mlfairness/income.pkl...
/ [1 files][  1.6 MiB/  1.6 MiB]                                                
Operation completed over 1 objects/1.6 MiB.                                      
Copying gs://cloud-training-demos/mlfairness/x_test.npy...
/ [1 files][572.5 KiB/572.5 KiB]                                                
Operation completed over 1 objects/572.5 KiB.                                    
Copying gs://cloud-training-demos/mlfairness/y_test.npy...
/ [1 files][ 63.7 KiB/ 63.7 KiB]                                                
Operation completed over 1 objects/63.7 KiB.                                     


In [32]:
features = pd.read_pickle('income.pkl')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

---
Now take a quick look at the data.  The ML model type used for this analysis is XGBoost.  XGBoost is a machine learning framework that uses decision trees and gradient boosting to build predictive models. It works by ensembling multiple decision trees together based on the score associated with different leaf nodes in a tree. 

XGBoost requires all values to be numeric so the orginial dataset was modified slightly.  The biggest change made was to assign a numeric value to Sex.  The originial dataset only had the values "Female" and "Male" for Sex.  The decision was made to assign the value "1" to Female and "2" to Male. As part of the data prepartion effort the Pandas function "get_dummies" was used to convert the remaining domain values into numerical equivalent.  For instance the "Education" column was turned into several sub-columns named after the value in the column.  For instance the "Education_HS-grad" has a value of "1" for when that was the orginial categorical value and a value of "0" for other cateogries.

In [33]:
features.head()

,Age,fnlwgt,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Workclass_ ?,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,Workclass_ Private,Workclass_ Self-emp-inc,Workclass_ Self-emp-not-inc,Workclass_ State-gov,Workclass_ Without-pay,Education_ 10th,Education_ 11th,Education_ 12th,Education_ 1st-4th,Education_ 5th-6th,Education_ 7th-8th,Education_ 9th,Education_ Assoc-acdm,Education_ Assoc-voc,Education_ Bachelors,Education_ Doctorate,Education_ HS-grad,Education_ Masters,Education_ Preschool,Education_ Prof-school,Education_ Some-college,Race_ Amer-Indian-Eskimo,Race_ Asian-Pac-Islander,Race_ Black,Race_ Other,Race_ White
16054,42,29297,2,0,0,40,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
32382,44,19099,1,0,0,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
10749,39,-5147,2,0,0,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
15377,39,16476,2,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
29660,29,-24893,1,0,1408,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


---
To connect the What-if Tool to an AI Platform model, you need to pass it a subset of your test examples.  The commannd below will create a Numpy array of 2000 from our test examples.

In [34]:
# Combine the features and labels into one array for the What-if Tool

num_wit_examples = 2000

test_examples = np.hstack((
    x_test[:num_wit_examples],
    y_test[:num_wit_examples].reshape(-1, 1)
))

---
Instantiating the What-if Tool is as simple as creating a WitConfigBuilder object and passing it the AI Platform model desired to be analyzed.

The optional "adjust_prediction" parameter is used because the What-if Tool expects a list of scores for each class in our model (in this case 2). Since the model only returns a single value from 0 to 1, it must be transformed to the correct format in this function.   Lastly, the name 'income_prediction' is used as the ground truth label.

It may take 1 to 2 minutes for the What-if Tool to load and render the visualization palette, please be patient.

In [35]:
PROJECT

'dherin-sandbox'

In [36]:
MODEL

'xgboost_model'

In [37]:
VERSION

'v1'

In [39]:
FEATURE_NAMES = features.columns.tolist() + ['income_prediction']


def adjust(pred):
    return [1 - pred, pred]


config_builder = (
    WitConfigBuilder(test_examples.tolist(), FEATURE_NAMES)
    .set_ai_platform_model(PROJECT, MODEL, VERSION, adjust_prediction=adjust)
    .set_target_feature('income_prediction')
    .set_label_vocab(['low', 'high'])
)

WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'classification', 'label_vocab': ['low', 'high'], 'feature_names': ['Age', 'fn…

---

## In the following steps we will use the What-if Tool to examine our model 

Since the What-If Tool is embedded in the notebook you must use care when scrolling.  The What-if Tool has its own internal scrolling windows so you may need to reposition the frame window to reach the desired location.  To do this you must ensure you are at the far edges of the cell to scroll up and down as noted by the red oblong markers in the image below.

Also, it might be helpful to widen the display frame in the notebook.  You can do this by dragging the vertical grey bar to the left (the place to click and hold is noted by the red arrows).

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/WIT_25.png "WIT UI")

---
The scenario for consideration is that this model, which is used to predict income levels, will be used in a loan approval process.   Your task is to determine its fitness for such a use case from a fairness perspective.

Your first action will be to examine data and its disribution along dimensions that are relevant to loan scoring.   The intial presentation in the tool shows all datapoints.  Blue dots are those individuals predicted as having incomes above 50k.  Red dots are those predicted as having incomes below 50k.

On the "Datapoint Editor" tab, under "Binning | X-Axis" select "Education_Masters":

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/BinX_Menu_Choice.png "BinX Menu")

The results show that a majority of Masters degree holders, domain value "1" (in red above the visualization), have incomes above 50k.

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/BinX_Results.png "BinX Results")

Return to the "Binning | X-Axis" selector and choose "None" to reset the visualization.

---

Next navigate to the Features tab, here you can see the exact distribution of values for every feature in the dataset.  If you type "sex" into the filter box, you will see that of the 2,000 test datapoints, 670 from Women and 1,330 are from Men (as mentioned earlier the value "1" was assigned to Females and "2" was assigned to Males). The dataset reflects an imbalance between Females and Males with nearly double the number of cases that are Male. Women seem under-represented in this dataset.

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/Features.png "Features")

On the "Performance + Fairness" tab, you can set an input feature (or set of features) by which to slice the data. This will allow you to evaluate the fairness of specific groups.  Income Prediction (corresponding to over or under 50k) has already been selected as the "Ground Truth Feature". On the "Slice by" selector, scroll to find and choose "Sex".

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/PerfTab.png "PerfTab")

This selection allows you to see the breakdown of model performance on female datapoints versus male datapoints.  Even before you drill into the details you can see that the model has a higher false accuracy value for females than males. Drilling down on each value (by clicking the arrow beside the domain value) you see that the model predicts high income for females much less than it does for males:  3.4% of the time for females vs 18.9% of the time for males.

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/MaleDetails.png "Male Details")

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/FemaleDetails.png "Female Details")

In the use-case scenario, the plan is to use this simple income classifier to approve or reject loan applications (not a realistic example but it illustrates the effect of bias in ML usage). In this case, 18.9% of men from the test dataset have their loans approved but only 3.4% of women have their loans approved. If you wanted to ensure than men and women get their loans approved the same percentage of the time, that is a fairness concept called "demographic parity". An XGBoost model intially defaults to a 0.50 threshold, which is what appears upon initial examination.  One way to achieve demographic parity would be to have different classification thresholds for females and males in our model. You'll notice there is a button on the tool labeled "demographic parity". When you press this button, the tool will take the cost ratio into account, and come up with ideal separate thresholds for men and women that will achieve demographic parity over the test dataset.

On the "Performance + Fairness" tab, select "Demographic Parity" to see the results.

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/DemoParityButton_Resize.png "Demographic Parity Button")

By drilling down on each domain value you can see the automatic adjustments.  In this case, demographic parity can be found with both groups getting loans approved/predicting a high income ~17.7% of the time.  This occurs when the female threshold is set to 0.19 and the male threshold is set to 0.54. Because of the vast difference in the properties of the female and male training data in this 1994 census dataset, you need quite different thresholds to achieve demographic parity. With the high male threshold you may notice there are many more false negatives than before, and with the low female threshold there are many more false positives than before.  To reset the Peformance and Fairness Tab simply choose another domain value in the "Slice by" selector.

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/MaleDemoParitySlider.png "Male Demo Parity")

![alt_text](https://storage.cloud.google.com/cloud-training-demos/mlfairness/images/FemaleDemoParitySlider.png "Female Demo Parity")




---
The use of these features can help shed light on subsets of your data on which your classifier is performing very differently. Understanding biases in your datasets and data slices on which your model has disparate performance are very important parts of analyzing a model for fairness. There are many approaches to improving fairness, including augmenting training data, building fairness-related loss functions into your model training procedure, and post-training inference adjustments like those seen in WIT. The WIT provides a great interface for furthering ML fairness learning, but of course there is no silver bullet to improving ML fairness.

Feel free to explore the What-if Tool and find additional insights.

This is the end of the lab.